In [1]:

import torch
import numpy as np

from mmengine.config import Config
from mmengine.registry import MODELS
from pathlib import Path
from geoai.data.utils import Image
from osgeo import gdal_array
import shutil
import tifffile

In [2]:
device = torch.device('cuda')

In [3]:
# load config
cfg = Config.fromfile('../../configs/cloud_masks/deeplab.py')

In [4]:
cfg['model']['data_preprocessor']['size'] = (1000, 1000)
cfg['model']['test_cfg']['mode'] = 'whole'

In [5]:
model = MODELS.build(cfg['model'])

/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/mmseg/models/backbones/resnet.py:431: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:

In [6]:
_ = model.to(device)

In [7]:
checkpoint = '/home/sandeep/workspace/competitions/MaskingCloudsinSatelliteImageries/training/mmseg_test/iter_1080.pth'
model.load_state_dict(torch.load(checkpoint)['state_dict'])

<All keys matched successfully>

In [8]:
eval_images_path = '/home/sandeep/workspace/competitions/MaskingCloudsinSatelliteImageries/data/evaluation_true_color'
eval_masks_path = '/home/sandeep/workspace/competitions/MaskingCloudsinSatelliteImageries/evaluation_mask'

eval_images_path = Path(eval_images_path)
eval_masks_path = Path(eval_masks_path)
if eval_masks_path.exists():
    shutil.rmtree(eval_masks_path)
eval_masks_path.mkdir()

In [9]:
# mean = torch.tensor([2315.3093, 2460.9182, 2424.4849], dtype=torch.float32, device=device).view(1, -1, 1, 1)
# std = torch.tensor([1370.4095, 1372.0483, 1484.2986], dtype=torch.float32, device=device).view(1, -1, 1, 1)

In [10]:
model.eval()
eval_images = list(eval_images_path.glob('*.tif'))
for i, image in enumerate(eval_images):
    print(f"-- Progress: {i}/{len(eval_images)}", " "*5, end='\r')
    x = Image.from_path(image).data.astype(np.float32)[None]
    x = torch.tensor(x)
    x = model.data_preprocessor({'inputs': x})['inputs']
    with torch.no_grad():
        y = model.predict(x)[0].pred_sem_seg
    mask = y.data.cpu().numpy().squeeze().astype(np.uint8)
    out_file = eval_masks_path / f"evaluation_mask_{image.stem.split('_')[-1]}.tif"
    tifffile.imwrite(out_file, mask)
    # ds = gdal_array.OpenArray(mask)
    # Image.from_gdal(ds).save(out_file)
    # del ds

/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/mmseg/models/utils/wrappers.py:22: UserWarning: When align_corners=True, the output would more aligned if input size (125, 125) is `x+1` and out size (250, 250) is `nx+1`
  warnings.warn(
/home/sandeep/miniconda3/envs/geoai/lib/python3.10/site-packages/mmseg/models/utils/wrappers.py:22: UserWarning: When align_corners=True, the output would more aligned if input size (250, 250) is `x+1` and out size (1000, 1000) is `nx+1`
  warnings.warn(
